In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.39
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`````

In [3]:
can = qa.QA_fetch_stock_day_adv('000002', start='2017-06-01', end='2018-05-29').to_qfq()

In [4]:
raw = can.data.loc[:, ['open', 'close', 'high', 'low']].values

In [6]:
dates = can.data.date

In [7]:
from pyecharts import Grid, Bar, Line, Kline, Overlap

kline = Kline()
kline.add("日K", dates.tolist(), raw, is_datazoom_show=True)

overlap = Overlap()
overlap.add(kline)
overlap

In [9]:
mrg = []
mrg_cnt = []
mrg_date = []
raw = raw
raw_date = dates.tolist()
raw_idx = 0
mrg_idx = 0

while True:
    if raw_idx >= len(raw):
        break
    if raw_idx == 0:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        raw_idx += 1
        continue
    if (mrg[mrg_idx][2] >= raw[raw_idx][2] and mrg[mrg_idx][3] <= raw[raw_idx][3]) or (mrg[mrg_idx][2] <= raw[raw_idx][2] and mrg[mrg_idx][3] >= raw[raw_idx][3]):
        if mrg_idx == 0 or mrg[mrg_idx-1][2] <= mrg[mrg_idx][2]:
            mrg[mrg_idx][0] = max(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = max(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = max(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = max(mrg[mrg_idx][3], raw[raw_idx][3])
        else:
            mrg[mrg_idx][0] = min(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = min(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = min(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = min(mrg[mrg_idx][3], raw[raw_idx][3])
        mrg_date[mrg_idx] = raw_date[raw_idx]
        mrg_cnt[mrg_idx] += 1
        raw_idx += 1
        continue
    mrg.append(raw[raw_idx])
    mrg_date.append(raw_date[raw_idx])
    mrg_cnt.append(1)
    raw_idx += 1
    mrg_idx += 1


In [27]:
kline = Kline()
kline.add("日K", mrg_date, mrg, is_datazoom_show=True, 
          mark_point=[{'coord':[mrg_date[-10], mrg[-10][2] * 1.03], 'name':'test'}], 
          mark_point_symbol='circle', mark_point_symbolsize=10)
kline
#overlap = Overlap()
#overlap.add(kline)
#overlap

In [ ]:
def higher(x, y):
    return x[2] >= y[2] and x[3] >= y[3]

def lower(x, y):
    return x[2] <= y[2] and x[3] <= y[3]

end = []
for idx, cell in enurmate(mrg[2:]):
    if higher(mrg[idx-1], mrg[idx-2]) and higher(mrg[idx-1], cell) and (idx-2 > last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last):
        end.append([idx-1, 1, True])
        last = idx
    elif lower(mrg[idx-1], mrg[idx-2]) and lower(mrg[idx-1], cell) and (idx-2 > last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last):
        end.append([idx-1, 0, True])
        last = idx
    if len(end)>1 and end[-1][1] == end[-2][1]:
        end[-2][2] = False
    